In [80]:
import utilities
import os
import socket
import tldextract
import pandas as pd
import csv

In [81]:
base_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Traffic"
dns = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/post_processed/encrypted_echo/general_dns_resolutions.json")
categories_echo = utilities.get_directories_in_a_directory("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Traffic/Traffic_echo_1.1.1.1/")

#### create a table of category | skill_name | FQDN | eSLD | subdomains |organization | ATSes | party

In [ ]:
# 224.0.0.251 : reserved for multicast DNS (mDNS)
# 255.255.255.255 multicast
# 239.255.255.250 Multicast address

In [97]:
df = {'cat': [], 'skill_name': [], 'fqdn': [], 'domain_name': [], 'SLD': [], 'subdomain' : [] ,'organization': [], 'ATSes': [], 'Party': []}
for category in categories_echo:
    category_name = category.split('/')[-1]
    jsons = utilities.get_files_in_a_directory(os.path.join(category, "jsons"))
    for skill in jsons:
        ip_list = set()
        skill_name = skill.split('/')[-1].split('.')[0]
        skill_network_json = utilities.read_json(skill)
        for i in range(len(skill_network_json)):
            if 'ip' in skill_network_json[i]["_source"]["layers"].keys():
                ip_src =  skill_network_json[i]["_source"]["layers"]['ip']['ip.src']
                ip_dst =   skill_network_json[i]["_source"]["layers"]['ip']['ip.dst']
                if not ip_src.startswith("10.42.") and ip_src != "224.0.0.251" and ip_src != "255.255.255.255" and ip_src != "239.255.255.250":
                    if ip_src not in ip_list:
                        ip_list.add(ip_src)
                        df['cat'].append(category_name)
                        df['skill_name'].append(skill_name)
                        if ip_src in dns:
                            df['fqdn'].append(dns[ip_src])
                            ext = tldextract.extract(dns[ip_src])
                            df['domain_name'].append(ext.domain+"."+ext.suffix)
                            df['subdomain'].append(ext.subdomain)
                            df['SLD'].append(ext.domain)
                            
                        else:
                            df['fqdn'].append(ip_src)
                            df['domain_name'] = ""
                            df['subdomain'].append("")
                            df['SLD'].append("")
                        df['organization'].append("")
                        df['ATSes'].append("")
                        df['Party'].append("")
                        
                if not ip_dst.startswith("10.42.") and ip_dst != "224.0.0.251" and ip_dst != "255.255.255.255" and ip_dst != "239.255.255.250":
                    if ip_dst not in ip_list:
                        ip_list.add(ip_dst)
                        df['cat'].append(category_name)
                        df['skill_name'].append(skill_name)
                        if ip_dst in dns:
                            df['fqdn'].append(dns[ip_dst])
                            ext = tldextract.extract(dns[ip_dst])
                            df['domain_name'].append(ext.domain+"."+ext.suffix)
                            df['subdomain'].append(ext.subdomain)
                            df['SLD'].append(ext.domain)
                        else:
                            df['fqdn'].append(ip_dst)
                            df['domain_name'].append("")
                            df['subdomain'].append("")
                            df['SLD'].append("")
                        df['organization'].append("")
                        df['ATSes'].append("")
                        df['Party'].append("")

# Save dataframe in csv format

In [98]:
df = pd.DataFrame.from_dict(df)
df.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report.csv", sep=',')
df

,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
0,Fashion-Style,B083PY1PGZ,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,,,
1,Fashion-Style,B083PY1PGZ,ec2-54-166-14-123.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-54-166-14-123.compute-1,,,
2,Fashion-Style,B087YM4KJK,mlis.amazon.com,amazon.com,amazon,mlis,,,
3,Fashion-Style,B087YM4KJK,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,,,
4,Fashion-Style,B087YM4KJK,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,,,
...,...,...,...,...,...,...,...,...,...
2600,Health-Fitness,B07PJWHYXM,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,,,
2601,Health-Fitness,B07PJWHYXM,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,,,
2602,Health-Fitness,B07PJWHYXM,ec2-34-198-99-173.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-34-198-99-173.compute-1,,,
2603,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,,,


In [42]:
df = pd.read_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report.csv")
df

,Unnamed: 0,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
0,0,Fashion-Style,B083PY1PGZ,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,"Amazon Technologies, Inc.",NaN,NaN
1,1,Fashion-Style,B083PY1PGZ,ec2-54-166-14-123.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-54-166-14-123.compute-1,"Amazon Technologies, Inc.",NaN,NaN
2,2,Fashion-Style,B087YM4KJK,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",NaN,NaN
3,3,Fashion-Style,B087YM4KJK,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,"Amazon Technologies, Inc.",NaN,NaN
4,4,Fashion-Style,B087YM4KJK,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,"Amazon Technologies, Inc.",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2790,2790,Health-Fitness,B07PJWHYXM,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,"Amazon Technologies, Inc.",NaN,NaN
2791,2791,Health-Fitness,B07PJWHYXM,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,"Amazon Technologies, Inc.",NaN,NaN
2792,2792,Health-Fitness,B07PJWHYXM,ec2-34-198-99-173.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-34-198-99-173.compute-1,"Amazon Technologies, Inc.",NaN,NaN
2793,2793,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",NaN,NaN


# Unique Domain Names

In [120]:
unique_df = {}
for index, row in df.iterrows():
    if row['fqdn'] not in unique_df:
        unique_df[row['fqdn']] = {'domain_name': row['domain_name'], 'SLD': row['SLD'], 'subdomain' : row['subdomain'] ,'organization': "", 'ATSes': "", 'Party': ""}
unique_df

{'bob-dispatch-prod-na.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'bob-dispatch-prod-na',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'ec2-54-166-14-123.compute-1.amazonaws.com': {'domain_name': 'amazonaws.com',
  'SLD': 'amazonaws',
  'subdomain': 'ec2-54-166-14-123.compute-1',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'd1gsg05rq1vjdw.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd1gsg05rq1vjdw',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': '',
  'ATSes': '',
  'Party': ''},
 'play.podtrac.com': {'domain_name': 'podtrac.com',
  'SLD': 'podtrac',
  'subdomain': 'play',
  'organization': '',
  'ATSes'

## Address Organization

In [121]:
ddg_entities = utilities.get_files_in_a_directory("/home/c2/alexa/post-processing/tracker-radar/entities")

for fqdn, details in unique_df.items():
    for entity in ddg_entities:
        entity_content = utilities.read_json(entity)
        if details['domain_name'] in entity_content["properties"]:
            details['organization'] = entity_content["name"]

####  For organizations we didn't find on DDG, we use https://lookup.icann.org/lookup and crunchbase.com

In [122]:
non_org_in_ddg = {}
for fqdn, details in unique_df.items(): 
    if details['organization'] == '':
        if fqdn not in non_org_in_ddg:
            non_org_in_ddg[fqdn] = details['domain_name']
#utilities.write_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/non_org_in_ddg.json", dict(non_org_in_ddg))

In [123]:
org_lookup_domain = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/org_lookup_domain.json")
for fqdn, details in unique_df.items(): 
    if details['organization'] == '':
        if details['domain_name'] in org_lookup_domain:
            details['organization'] = org_lookup_domain[details['domain_name']]

In [124]:
for fqdn, details in unique_df.items(): 
    if details['organization'] == '':
        print(fqdn)

In [125]:
unique_df

{'bob-dispatch-prod-na.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'bob-dispatch-prod-na',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'ec2-54-166-14-123.compute-1.amazonaws.com': {'domain_name': 'amazonaws.com',
  'SLD': 'amazonaws',
  'subdomain': 'ec2-54-166-14-123.compute-1',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'd1gsg05rq1vjdw.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd1gsg05rq1vjdw',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': '',
  'Party': ''},
 

## Address ATSes

In [126]:
filter_list_base = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists/"
tracking = utilities.read_json(os.path.join(filter_list_base,"tracking", "tracking_list.json"))
advertising = utilities.read_json(os.path.join(filter_list_base, "advertising", "advertising_list.json"))

In [127]:
for fqdn, details in unique_df.items():
    if fqdn in tracking or details['domain_name'] in tracking:
        details['ATSes'] = "Tracking/Analytics"
        print("tracker", fqdn)
        continue
    if fqdn in advertising or details['domain_name'] in advertising:
        details['ATSes'] = "Ad"
        print("ad", fqdn)
        continue
    else:
        details['ATSes'] = 'others'
        continue

ad device-metrics-us-2.amazon.com
tracker chtbl.com
tracker pool-na-east-001-anycast.mc.tritondigital.com
tracker 24163.mc.tritondigital.com


In [128]:
unique_df

{'bob-dispatch-prod-na.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'bob-dispatch-prod-na',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'others',
  'Party': ''},
 'ec2-54-166-14-123.compute-1.amazonaws.com': {'domain_name': 'amazonaws.com',
  'SLD': 'amazonaws',
  'subdomain': 'ec2-54-166-14-123.compute-1',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'others',
  'Party': ''},
 'mlis.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'mlis',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'others',
  'Party': ''},
 'd1gsg05rq1vjdw.cloudfront.net': {'domain_name': 'cloudfront.net',
  'SLD': 'cloudfront',
  'subdomain': 'd1gsg05rq1vjdw',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes': 'others',
  'Party': ''},
 'device-metrics-us-2.amazon.com': {'domain_name': 'amazon.com',
  'SLD': 'amazon',
  'subdomain': 'device-metrics-us-2',
  'organization': 'Amazon Technologies, Inc.',
  'ATSes

In [129]:
df

,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
0,Fashion-Style,B083PY1PGZ,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,,others,non_Platform-based
1,Fashion-Style,B083PY1PGZ,ec2-54-166-14-123.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-54-166-14-123.compute-1,,others,non_Platform-based
2,Fashion-Style,B087YM4KJK,mlis.amazon.com,amazon.com,amazon,mlis,,others,non_Platform-based
3,Fashion-Style,B087YM4KJK,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,,others,non_Platform-based
4,Fashion-Style,B087YM4KJK,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,,others,non_Platform-based
...,...,...,...,...,...,...,...,...,...
2600,Health-Fitness,B07PJWHYXM,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,,others,non_Platform-based
2601,Health-Fitness,B07PJWHYXM,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,,others,non_Platform-based
2602,Health-Fitness,B07PJWHYXM,ec2-34-198-99-173.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-34-198-99-173.compute-1,,others,non_Platform-based
2603,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,,others,non_Platform-based


### Edit Final DataFrame

In [130]:
for index, row in df.iterrows():
    df.loc[index, 'organization'] = unique_df[row['fqdn']]['organization']

In [131]:
#organization = set()
for index, row in df.iterrows():
    if row['organization'] == 'Triton Digital ':
        df.loc[index, 'organization'] = 'Triton Digital, Inc.' 
    df.loc[index, 'ATSes'] = unique_df[row['fqdn']]['ATSes']
    if row['organization'] == 'Amazon Technologies, Inc.':
        df.loc[index, 'Party'] = "Platform-based"
        continue
    else:
        df.loc[index, 'Party'] = "non_Platform-based"

In [132]:
df = pd.DataFrame.from_dict(df)
df.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report.csv", sep=',')
df

,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
0,Fashion-Style,B083PY1PGZ,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,"Amazon Technologies, Inc.",others,Platform-based
1,Fashion-Style,B083PY1PGZ,ec2-54-166-14-123.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-54-166-14-123.compute-1,"Amazon Technologies, Inc.",others,Platform-based
2,Fashion-Style,B087YM4KJK,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",others,Platform-based
3,Fashion-Style,B087YM4KJK,bob-dispatch-prod-na.amazon.com,amazon.com,amazon,bob-dispatch-prod-na,"Amazon Technologies, Inc.",others,Platform-based
4,Fashion-Style,B087YM4KJK,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,"Amazon Technologies, Inc.",others,Platform-based
...,...,...,...,...,...,...,...,...,...
2600,Health-Fitness,B07PJWHYXM,d3p8zr0ffa9t17.cloudfront.net,cloudfront.net,cloudfront,d3p8zr0ffa9t17,"Amazon Technologies, Inc.",others,Platform-based
2601,Health-Fitness,B07PJWHYXM,d1gsg05rq1vjdw.cloudfront.net,cloudfront.net,cloudfront,d1gsg05rq1vjdw,"Amazon Technologies, Inc.",others,Platform-based
2602,Health-Fitness,B07PJWHYXM,ec2-34-198-99-173.compute-1.amazonaws.com,amazonaws.com,amazonaws,ec2-34-198-99-173.compute-1,"Amazon Technologies, Inc.",others,Platform-based
2603,Health-Fitness,B07PJWHYXM,mlis.amazon.com,amazon.com,amazon,mlis,"Amazon Technologies, Inc.",others,Platform-based


### Thir party Analysis

In [141]:
df = pd.read_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/report.csv")
df_3rd = df[df['Party'] == 'non_Platform-based']
del df_3rd['Unnamed: 0']
df_3rd.groupby('cat').count()

,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
cat,,,,,,,,
ConnectedCar,30,30,30,30,23,30,30,30
Dating,19,19,19,19,14,19,19,19
Fashion-Style,19,19,19,19,14,19,19,19
Health-Fitness,20,20,20,20,20,20,20,20
Navigation-TripPlanners,7,7,7,7,5,7,7,7
Pets-Animals,16,16,16,16,14,16,16,16
Religion-Spirituality,8,8,8,8,8,8,8,8
Wine-Beverages,3,3,3,3,3,3,3,3


In [142]:
df_3rd

,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
19,Fashion-Style,B01LWX3TFV,play.podtrac.com,podtrac.com,podtrac,play,"Domains By Proxy, LLC",others,non_Platform-based
79,Fashion-Style,B073WHY36J,chtbl.com,chtbl.com,chtbl,NaN,Chartable Holding Inc,Tracking/Analytics,non_Platform-based
141,Fashion-Style,B07CG1ZPY6,traffic.omny.fm,omny.fm,omny,traffic,"Triton Digital, Inc.",others,non_Platform-based
179,Fashion-Style,B072N6BNB1,play.podtrac.com,podtrac.com,podtrac,play,"Domains By Proxy, LLC",others,non_Platform-based
180,Fashion-Style,B072N6BNB1,chtbl.com,chtbl.com,chtbl,NaN,Chartable Holding Inc,Tracking/Analytics,non_Platform-based
...,...,...,...,...,...,...,...,...,...
2553,Health-Fitness,B01MRM361G,1920044715.rsc.cdn77.org,cdn77.org,cdn77,1920044715.rsc,DataCamp Limited,others,non_Platform-based
2565,Health-Fitness,B018WUNBE6,1920044715.rsc.cdn77.org,cdn77.org,cdn77,1920044715.rsc,DataCamp Limited,others,non_Platform-based
2588,Health-Fitness,B076TVD85K,1920044715.rsc.cdn77.org,cdn77.org,cdn77,1920044715.rsc,DataCamp Limited,others,non_Platform-based
2594,Health-Fitness,B07QN179C5,1920044715.rsc.cdn77.org,cdn77.org,cdn77,1920044715.rsc,DataCamp Limited,others,non_Platform-based


In [148]:
df_3rd_tritondigital = df_3rd[df_3rd['SLD'] == 'tritondigital']
df_3rd_tritondigital.groupby('cat').size()
df_3rd_tritondigital

,cat,skill_name,fqdn,domain_name,SLD,subdomain,organization,ATSes,Party
182,Fashion-Style,B072N6BNB1,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
183,Fashion-Style,B072N6BNB1,24163.mc.tritondigital.com,tritondigital.com,tritondigital,24163.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
257,Fashion-Style,B07B6FLP91,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
261,Fashion-Style,B07B6FLP91,24163.mc.tritondigital.com,tritondigital.com,tritondigital,24163.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
322,ConnectedCar,B079NYGY7G,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
324,ConnectedCar,B079NYGY7G,24163.mc.tritondigital.com,tritondigital.com,tritondigital,24163.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
478,ConnectedCar,B075KQ35RL,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
479,ConnectedCar,B075KQ35RL,24163.mc.tritondigital.com,tritondigital.com,tritondigital,24163.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
612,ConnectedCar,B075TRB4V5,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based
729,ConnectedCar,B08DQZ9869,pool-na-east-001-anycast.mc.tritondigital.com,tritondigital.com,tritondigital,pool-na-east-001-anycast.mc,"Triton Digital, Inc.",Tracking/Analytics,non_Platform-based


In [158]:
subgroup_skills = utilities.read_json("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/skills_data/subgrouped_skills.json")
df_3rd_tritondigital['skill_link'] = ""
for index, row in df_3rd_tritondigital.iterrows():
    df_3rd_tritondigital.loc[index, 'skill_link'] = subgroup_skills[row['cat']][row['skill_name']]['Skill_link'] 
df_3rd_tritondigital.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Rahmadi/reporttritondigital_.csv", sep=',')

/tmp/ipykernel_89527/3485130117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3rd_tritondigital['skill_link'] = ""
/home/c2/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/c2/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [161]:
df_3rd_podtrac = df_3rd[df_3rd['SLD'] == 'podtrac']
df_3rd_podtrac.groupby('cat').size()
df_3rd_podtrac

df_3rd_podtrac['skill_link'] = ""
for index, row in df_3rd_podtrac.iterrows():
    df_3rd_podtrac.loc[index, 'skill_link'] = subgroup_skills[row['cat']][row['skill_name']]['Skill_link'] 
df_3rd_podtrac.to_csv("/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Rahmadi/report-podtrac.csv", sep=',')

/tmp/ipykernel_89527/1031774145.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3rd_podtrac['skill_link'] = ""
/home/c2/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/c2/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [162]:
for index, row in df_3rd_podtrac.iterrows():
    file_name = row['skill_name'] + ".PCAPS"
    categoty = row['cat']
    

B01LWX3TFV.PCAPS
B072N6BNB1.PCAPS
B07MYH64DY.PCAPS
B07B6FLP91.PCAPS
B07B6FLP91.PCAPS
B079NYGY7G.PCAPS
B079NYGY7G.PCAPS
B084GHYRK6.PCAPS
B075KQ35RL.PCAPS
B088NK1XK3.PCAPS
B075TRB4V5.PCAPS
B08DQZ9869.PCAPS
B08DQZ9869.PCAPS
B07HBNT5QM.PCAPS
B07G75Y7TZ.PCAPS
B083LRV775.PCAPS
B083LRV775.PCAPS
B083LRV775.PCAPS
B084RS2L6C.PCAPS
B085L7SV2B.PCAPS
B085L7SV2B.PCAPS
